In [ ]:
!pip install squanch

In [ ]:
from squanch import *
import numpy as np
import matplotlib.image as image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class Charlie(Agent):

    def run(self):
        for qsys in self.qstream:
            a, b = qsys.qubits
            H(a)
            CNOT(a, b)
            self.qsend(alice, a)
            self.qsend(bob, b)

In [ ]:
class Alice(Agent):

    def run(self):
        for _ in self.qstream:
            bit1 = self.data.pop(0)
            bit2 = self.data.pop(0)
            q = self.qrecv(charlie)
            if q is not None:
                if bit2 == 1: X(q)
                if bit1 == 1: Z(q)
            self.qsend(eve, q) # Alice unknowingly sends the qubit to Eve

In [ ]:
class Bob(Agent):

    def run(self):
        bits = []
        for _ in self.qstream:
            a = self.qrecv(eve) # Bob receives his qubit from Eve
            c = self.qrecv(charlie)
            if a is not None and c is not None:
                CNOT(a, c)
                H(a)
                bits.extend([a.measure(), c.measure()])
            else:
                bits.extend([0,0])
        self.output(bits)

In [ ]:
class Eve(Agent):
    '''Eve naively tries to intercept Alice's message'''
    def run(self):
        bits = []
        for _ in self.qstream:
            a = self.qrecv(alice)
            if a is not None:
                bits.append(a.measure())
            else:
                bits.append(0)
            self.qsend(bob, a)
        self.output(bits)

In [ ]:
# Load an image and serialize it to a bitstream
img = image.imread("../docs/source/img/foundryLogo.bmp")
bitstream = list(np.unpackbits(img))

# Make the QStream for the agents to operate on
qstream = QStream(2, int(len(bitstream) / 2))

# Make agent instances
out = Agent.shared_output()

alice = Alice(qstream, out, data = bitstream)
bob = Bob(qstream, out)
charlie = Charlie(qstream, out)
eve = Eve(qstream, out)

# Connect the agents over simulated fiber optic lines
alice.qconnect(bob)
alice.qconnect(eve)
alice.qconnect(charlie)
bob.qconnect(charlie)
bob.qconnect(eve)

NameError: ignored

In [ ]:
from squanch import *
results = [] # Where we'll put the measurement results
for _ in range(10):
qsys = QSystem(2)
a, b = qsys.qubits # enumerate the qubits of the system
# Make a Bell pair
H(a)
CNOT(a, b)
# Measure the pair and append to results
results.append([a.measure(), b.measure()])
print(results)

IndentationError: ignored